# Manual QPE construction for Mock qDrift

In [200]:
import numpy as np
import scipy
import math

In [201]:
U = 2
t = 1
a = U + t
time = 1
N = 10000
hubbard = np.array([[U, -t], [-t, U]])


In [202]:
u_term = scipy.linalg.expm((1j * a * time / N) * np.array([[1, 0], [0, 1]]))
t_term = scipy.linalg.expm((1j * a * time / N) * np.array([[0, -1], [-1, 0]]))

In [203]:
print(u_term)

[[0.99999996+0.0003j 0.        +0.j    ]
 [0.        +0.j     0.99999996+0.0003j]]


In [208]:
def phase_estimation(unitary, negative: bool):
    """
    Quantum phase estimation
    Args:
        unitary: The unitary to use
        negative: if True, we assume the phase is negative and apply -2pi 
    Returns: The phase of the unitary operator

    """
    init_state = np.matrix([1, 0, 0, 0]).T
    hd = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    state_1 = np.dot(np.kron(hd, hd), init_state)
    
    c_unitary = np.zeros((4, 4), dtype = 'complex_')
    c_unitary[0, 0] = 1
    c_unitary[1, 1] = 1
    c_unitary[2, 2] = unitary[0, 0]
    c_unitary[2, 3] = unitary[0, 1]
    c_unitary[3, 2] = unitary[1, 0]
    c_unitary[3, 3] = unitary[1, 1]
    
    state_2 = np.dot(c_unitary, state_1)
    state_3 = np.dot(np.kron(hd, np.identity(2)), state_2)
    result = np.dot((1/ np.sqrt(2)) * np.matrix([1, 1, 0, 0]), state_3)
    prob = np.linalg.norm(result)
    phase = 2 * np.arccos(prob)
    if negative:
        phase = 2 * np.arccos(-1 * prob) - 2 * np.pi
    
    return phase

## Sample the phase estimation result

In [211]:
total = 0
result = []
for _ in range(N):
    choice = np.random.choice(np.array([0, 1]), p=[U/(U+t), t/(U+t)])
   
    if choice == 0:
        
        phase_result = phase_estimation(u_term, False)
        total += phase_result
        result.append(0)
    if choice == 1:
        
        phase_result = phase_estimation(t_term, True)
        total += phase_result
        result.append(1)
        
print(total)
    
        

1.0002000127042998


In [212]:
# Checks the SAMPLE() distribution is correct
print(result.count(0))

6667


### Testing the phase estimation code

In [196]:
init_state = np.matrix([1, 0, 0, 0]).T
hd = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])
state_1 = np.dot(np.kron(hd, hd), init_state)
print(f"state_1: {state_1}")
unitary = scipy.linalg.expm(-1j * np.pi/ 1000 * np.identity(2))
# unitary = t_term
c_unitary = np.zeros((4, 4), dtype = 'complex_')
c_unitary[0, 0] = 1
c_unitary[1, 1] = 1
c_unitary[2, 2] = unitary[0, 0]
c_unitary[2, 3] = unitary[0, 1]
c_unitary[3, 2] = unitary[1, 0]
c_unitary[3, 3] = unitary[1, 1]
print(c_unitary)
state_2 = np.dot(c_unitary, state_1)
print(f"state_2: {state_2}")
state_3 = np.dot(np.kron(hd, np.identity(2)), state_2)
print(f"state_3: {state_3}")
result = np.dot((1/ np.sqrt(2)) * np.matrix([1, 1, 0, 0]), state_3)
print(result)
prob = np.linalg.norm(result)
print(prob)
print(2 * np.arccos(-prob)-2*np.pi)
print(math.degrees(2 * np.arccos(-prob))-360)

state_1: [[0.5]
 [0.5]
 [0.5]
 [0.5]]
[[1.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         1.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.99999507-0.00314159j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.99999507-0.00314159j]]
state_2: [[0.5       +0.j        ]
 [0.5       +0.j        ]
 [0.49999753-0.00157079j]
 [0.49999753-0.00157079j]]
state_3: [[7.07105036e-01-0.00111072j]
 [7.07105036e-01-0.00111072j]
 [1.74471461e-06+0.00111072j]
 [1.74471461e-06+0.00111072j]]
[[0.99999753-0.00157079j]]
0.9999987662997032
-0.0031415926540265815
-0.18000000002501793
